In [1]:
import pandas as pd
import numpy as np
import napari
import pytest

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection
import blimp.processing.quantify

29-Oct-24 18:55:27 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
viewer = napari.Viewer()

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:

_data_dir = Path("/Users/z3532965/src/blimp/tests/_data/")
intensity_image_2D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_intensity_image_TYX.tiff")
label_image_2D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_label_image_TYX.tiff")
intensity_image_3D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_intensity_image_TZYX.tiff")
label_image_3D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_label_image_TZYX.tiff")




``_measure_parent_object_label``

In [5]:
with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    blimp.processing.quantify._measure_parent_object_label(
        label_image = label_image_2D,
        measure_object_index = 2,
        parent_object_index = 0,
        timepoint = 0)

res = blimp.processing.quantify._measure_parent_object_label(
    label_image = label_image_2D,
    measure_object_index = 1,
    parent_object_index = 0,
    timepoint = 0)

res.to_csv(_data_dir / "datasets" / "synthetic_images" / "measure_parent_label_2D_results_t_0.csv")

[viewer.add_labels(label_image_2D.get_image_data("YX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


[<Labels layer 'Object1' at 0x34160b040>,
 <Labels layer 'Object2' at 0x341636ef0>,
 <Labels layer 'Object3' at 0x346559300>]

In [6]:
# check visually that this looks ok

# Convert DataFrame to dictionary for mapping
parent_labels_dict = dict(zip(res['label'], res['parent_label']))

# Replace values in the 2D array using np.vectorize with the dictionary
original_labels_array = label_image_2D.get_image_data("YX",C=1)
parent_labels_array = np.vectorize(parent_labels_dict.get)(original_labels_array, original_labels_array)


In [7]:
viewer.add_labels(parent_labels_array)

<Labels layer 'parent_labels_array' at 0x345d26dd0>

3D

In [23]:
res = blimp.processing.quantify._measure_parent_object_label(
    label_image = label_image_3D,
    measure_object_index = 1,
    parent_object_index = 0,
    timepoint = 0)
res

,label,parent_label,parent_label_name


``_quantify_single_timepoint_2D``

In [8]:
# no parent object
res_obj1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0)

res_obj1.Object1_intensity_max_Channel1.to_list() == [1000.,2000.,3000.,4000.]
res_obj1.Object1_area.to_list() == [100.0,81.0,900.0,961.0]

res_obj2 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0)

res_obj2.count().label==100
list(np.unique(res_obj2.Object2_area)) == [4.0]

with pytest.raises(AttributeError):
    # check no parent label when parent_label is none
    parents = res_obj2.parent_label

# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    res_obj3_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
        intensity_image = intensity_image_2D,
        label_image = label_image_2D,
        measure_object = 2,
        parent_object = 0,
        timepoint = 0)

# quantify object channel 1 relative to parent object channel 0
res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    parent_object = 0,
    timepoint = 0)

list(np.unique(res_obj2_parent1.parent_label_name)) == ["Object1"]

# check the number of objects in each parent object is correct
res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list() == [5,4,21,20]


True

In [9]:
# test intensity_channels
res_obj2_intensity1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels="Channel1")

res_obj2_intensity1_list1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1"])

res_obj2_intensity1_list2_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1","Channel2"])

res_obj2_intensity1_list_int_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=[0,1])

with pytest.raises(AttributeError):
    res_obj2_intensity1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel3

list(np.unique(res_obj2_intensity1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel2)) == [0,5000.]


True

In [10]:
# test texture_channels

# calculate_textures = False overrides texture channels provided
res_obj1_texture1_str_input_texture_false = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=False,
    texture_channels="Channel1")

with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel1_Haralick-angular-second-moment-1']

# calculate_textures = True with texture channels not specified calculates all
res_obj1_texture_none_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True)

len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

# calculate_textures = True with texture channels specified calculates only those specified
res_obj1_texture1_str_input_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True,
    texture_channels="Channel1")

res_obj1_texture1_str_input_texture_true['Object1_Channel1_Haralick-angular-second-moment-1'].to_list() == [1.,1.,1.,1.]
with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel3_Haralick-angular-second-moment-1']


In [11]:
len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

True

In [12]:
# test no border objects
res_no_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border.Object1_is_border)

# test one border object (crop image to generate a border object)
res_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = AICSImage(
        intensity_image_2D.data[:,:,:,52:,52:],
        channel_names=intensity_image_2D.channel_names),
    label_image = AICSImage(
        label_image_2D.data[:,:,:,52:,52:],
        channel_names=label_image_2D.channel_names),
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

res_border.query("label==1").Object1_is_border


0    True
Name: Object1_is_border, dtype: bool

``_quantify_single_timepoint_3D``

In [13]:
label_image_3D.channel_names
intensity_image_3D.channel_names

['Channel1', 'Channel2', 'Channel3']

In [18]:
# no parent object

res_obj1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    timepoint = 0)

res_obj1['Object1-3D-MIP_area'].to_list()==[10.**2, 9.**2, 30.**2, 46.**2]
res_obj1['Object1_3D_area'].to_list()==[10.**3, 9.**3, 30.**3, 46.**3]
res_obj1['Object1-3D-Middle_perimeter'].to_list()==[10.*4-4, 9.*4-4, 30.*4-4, 46.*4-4]

res_obj2 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0)

res_obj2.count().label==100
list(np.unique(res_obj2.Object2_3D_area)) == [2.**3]

with pytest.raises(AttributeError):
    # check no parent label when parent_label is none
    parents = res_obj2.parent_label

# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
#    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() #== [1,2,3,4]


AttributeError: 'DataFrame' object has no attribute 'parent_label'

In [21]:
res_obj2.columns

Index(['label', 'Object2_3D_centroid_0', 'Object2_3D_centroid_1',
       'Object2_3D_centroid_2', 'Object2_3D_area', 'Object2_3D_area_convex',
       'Object2_3D_axis_major_length', 'Object2_3D_axis_minor_length',
       'Object2_3D_extent', 'Object2_3D_feret_diameter_max',
       'Object2_3D_solidity', 'Object2_3D_intensity_mean_Channel3',
       'Object2_3D_intensity_max_Channel3',
       'Object2_3D_intensity_min_Channel3', 'Object2_3D_intensity_sd_Channel3',
       'Object2_3D_intensity_median_Channel3', 'Object2-3D-MIP_area',
       'Object2-3D-MIP_area_convex', 'Object2-3D-MIP_axis_major_length',
       'Object2-3D-MIP_axis_minor_length', 'Object2-3D-MIP_eccentricity',
       'Object2-3D-MIP_extent', 'Object2-3D-MIP_feret_diameter_max',
       'Object2-3D-MIP_solidity', 'Object2-3D-MIP_perimeter',
       'Object2-3D-MIP_perimeter_crofton', 'Object2-3D-MIP_euler_number',
       'Object2-3D-MIP_intensity_mean_Channel1',
       'Object2-3D-MIP_intensity_max_Channel1',
       'Object

In [17]:
res_obj2

,label,Object2_3D_centroid_0,Object2_3D_centroid_1,Object2_3D_centroid_2,Object2_3D_area,Object2_3D_area_convex,Object2_3D_axis_major_length,Object2_3D_axis_minor_length,Object2_3D_extent,Object2_3D_feret_diameter_max,...,Object2-3D-Middle_intensity_sd_Channel2,Object2-3D-Middle_intensity_median_Channel2,Object2-3D-Middle_intensity_mean_Channel3,Object2-3D-Middle_intensity_max_Channel3,Object2-3D-Middle_intensity_min_Channel3,Object2-3D-Middle_intensity_sd_Channel3,Object2-3D-Middle_intensity_median_Channel3,Object2_3D_is_border,Object2_is_border_XY,TimepointID
0,1,4.5,29.5,135.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,1356.465997,0.0,3200.000000,5000.0,0.0,2400.000000,5000.0,False,False,1
1,2,5.5,5.5,9.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,0.000000,0.0,3629.629630,6000.0,0.0,2933.183685,6000.0,False,False,1
2,3,5.5,6.5,5.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,332.591768,0.0,6097.777778,7000.0,0.0,2345.538449,7000.0,False,False,1
3,4,5.5,10.5,7.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,0.000000,0.0,7319.470699,8000.0,0.0,2231.841006,8000.0,False,False,1
4,5,6.5,9.5,10.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,170.5,37.5,22.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1
96,97,173.5,142.5,185.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1
97,98,185.5,81.5,138.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1
98,99,188.5,115.5,163.5,8.0,8.0,2.236068,2.236068,1.0,2.44949,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,1


In [ ]:
[viewer.add_labels(label_image_3D.get_image_data("ZYX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


In [ ]:
# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    res_obj3_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
        intensity_image = intensity_image_2D,
        label_image = label_image_2D,
        measure_object = 2,
        parent_object = 0,
        timepoint = 0)

# quantify object channel 1 relative to parent object channel 0
res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    parent_object = 0,
    timepoint = 0)

list(np.unique(res_obj2_parent1.parent_label_name)) == ["Object1"]

# check the number of objects in each parent object is correct
res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list() == [5,4,21,20]

In [ ]:
# test intensity_channels
res_obj2_intensity1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels="Channel1")

res_obj2_intensity1_list1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1"])

res_obj2_intensity1_list2_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1","Channel2"])

res_obj2_intensity1_list_int_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=[0,1])

with pytest.raises(AttributeError):
    res_obj2_intensity1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel3

list(np.unique(res_obj2_intensity1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel2)) == [0,5000.]


In [ ]:
# test texture_channels

# calculate_textures = False overrides texture channels provided
res_obj1_texture1_str_input_texture_false = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=False,
    texture_channels="Channel1")

with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel1_Haralick-angular-second-moment-1']

# calculate_textures = True with texture channels not specified calculates all
res_obj1_texture_none_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True)

len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

# calculate_textures = True with texture channels specified calculates only those specified
res_obj1_texture1_str_input_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True,
    texture_channels="Channel1")

res_obj1_texture1_str_input_texture_true['Object1_Channel1_Haralick-angular-second-moment-1'].to_list() == [1.,1.,1.,1.]
with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel3_Haralick-angular-second-moment-1']


In [ ]:
len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

In [ ]:
# test no border objects
res_no_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border.Object1_is_border)

# test one border object (crop image to generate a border object)
res_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = AICSImage(
        intensity_image_2D.data[:,:,:,52:,52:],
        channel_names=intensity_image_2D.channel_names),
    label_image = AICSImage(
        label_image_2D.data[:,:,:,52:,52:],
        channel_names=label_image_2D.channel_names),
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

res_border.query("label==1").Object1_is_border


In [ ]:
np.max(label_image_2D.get_image_data('YX',C=2))

In [ ]:
features_non_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=False)

In [ ]:
len(features_non_aggregated)

In [ ]:
features_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=True)

In [ ]:
features_aggregated.columns